Data taken from : 

- https://www.kaggle.com/datasets/gauravduttakiit/smoker-status-prediction-using-biosignals/data (not used)

- https://www.kaggle.com/competitions/playground-series-s3e24/overview

About Dataset
Smoking has been proven to negatively affect health in a multitude of ways.Smoking has been found to harm nearly every organ of the body, cause many diseases, as well as reducing the life expectancy of smokers in general. As of 2018, smoking has been considered the leading cause of preventable morbidity and mortality in the world, continuing to plague the world’s overall health.

According to a World Health Organization report, the number of deaths caused by smoking will reach 10 million by 2030.

Evidence-based treatment for assistance in smoking cessation had been proposed and promoted. however, only less than one third of the participants could achieve the goal of abstinence. Many physicians found counseling for smoking cessation ineffective and time-consuming, and did not routinely do so in daily practice. To overcome this problem, several factors had been proposed to identify smokers who had a better chance of quitting, including the level of nicotine dependence, exhaled carbon monoxide (CO) concentration, cigarette amount per day, the age at smoking initiation, previous quit attempts, marital status, emotional distress, temperament and impulsivity scores, and the motivation to stop smoking. However, individual use of these factors for prediction could lead to conflicting results that were not straightforward enough for the physicians and patients to interpret and apply. Providing a prediction model might be a favorable way to understand the chance of quitting smoking for each individual smoker. Health outcome prediction models had been developed using methods of machine learning over recent years.

A group of scientists are working on predictive models with smoking status as the prediction target.Your task is to help them create a machine learning model to identify the smoking status of an individual using bio-signals

Dataset Description

| Feature | Description |
| --- | --- |
| age | 5-year intervals |
| height (cm) | Height in centimeters |
| weight (kg) | Weight in kilograms |
| waist (cm) | Waist circumference |
| eyesight (left) | Vision (left eye) |
| eyesight (right) | Vision (right eye) |
| hearing (left) | Hearing (left ear) |
| hearing (right) | Hearing (right ear) |
| systolic | Systolic blood pressure |
| relaxation | Diastolic (relaxation) blood pressure |
| fasting blood sugar | Fasting blood sugar |
| Cholesterol | Total cholesterol |
| triglyceride | Triglyceride |
| HDL | High-density lipoprotein cholesterol |
| LDL | Low-density lipoprotein cholesterol |
| hemoglobin | Hemoglobin level |
| Urine protein | Urine protein level |
| serum creatinine | Serum creatinine |
| AST | Glutamic oxaloacetic transaminase |
| ALT | Glutamic pyruvic transaminase |
| Gtp | γ-GTP (Gamma-glutamyl transferase) |
| dental caries | Dental caries (yes/no) |
| smoking | Smoking status (target variable) |


# IMPORTS and LOAD data

In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from mini_eda_module import mini_eda # https://github.com/andyp14feb/miniEDAtoPDF


from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


df = pd.read_csv('./data/train.csv')
df = df.drop('id', axis=1)


# Function Colloection


## capping_features_based_on_threshold(df, thresholds_df)

In [ ]:
def capping_features_based_on_threshold(df, thresholds_df):
    for _, row in thresholds_df.iterrows():
        col = row['feature']
        min_val = row['min']
        max_val = row['max']
        
        flag_col = f"impossibleValue_{col}"
        df[flag_col] = ''
        
        below_flag = f"#{col}_belowMinThreshold; "
        above_flag = f"#{col}_aboveMaxThreshold; "
        
        df.loc[df[col] < min_val, flag_col] += below_flag
        df.loc[df[col] > max_val, flag_col] += above_flag
        
        df[col] = df[col].clip(lower=min_val, upper=max_val)
    
    return df

## calculate_IQR(df, column)

In [ ]:
def calculate_IQR(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identify outliers (data points outside the lower and upper bounds)
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    num_outliers = outliers.shape[0]
    percent_outliers = (num_outliers / len(df)) * 100
    above_upper = df[df[column] > upper_bound].shape[0]
    below_lower = df[df[column] < lower_bound].shape[0]
    
    return lower_bound, upper_bound, num_outliers, percent_outliers, above_upper, below_lower

## clip_values_based_on_IQR(df, column)

In [ ]:
def clip_values_based_on_IQR(df, column):
    # Calculate IQR, lower and upper bounds, and outliers
    lower_bound, upper_bound, _, _, _, _ = calculate_IQR(df, column)
    
    # Clip values outside the lower and upper bounds
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    
    return df

## show_corrMap(pandas_df, savedImageName)

In [ ]:
def show_corrMap(pandas_df, savedImageName):
    df_corr = pandas_df.corr(numeric_only=True)
    # --- create upper-triangle mask to avoid duplicates (optional) ---
    mask = np.triu(np.ones_like(df_corr, dtype=bool))

    plt.figure(figsize=(11, 9))
    sns.heatmap(
        df_corr,
        # mask=mask,              # comment this line if you want full matrix
        cmap='coolwarm',
        center=0,
        vmin=-1, vmax=1,
        annot=True,             # <-- numbers on cells
        fmt=".2f",              # 2-decimal format
        annot_kws={"size": 8},  # font size for numbers
        linewidths=.5,
        square=True,
        cbar_kws={"shrink": .8}
    )
    plt.title('Feature Correlation Matrix (annotated)', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(savedImageName, dpi=300)
    plt.show()

## add_combined_features(dfnya)

In [ ]:
def add_combined_features(dfnya):
    #kurus/gemuk (ukuran kegemukan)
    dfnya['new_bmi'] = (dfnya['weight_kg'] / (dfnya['height_cm'] ** 2))*100

    #lingkarpinggang (ukuran perut)
    dfnya['new_waist_height_ratio']=dfnya['waist_cm'] / dfnya['height_cm']

    #kekuatan pompa jantung
    dfnya['new_pulse_preasure'] = dfnya['systolic']-dfnya['relaxation']

    #penglihatan_rata-rata
    dfnya['new_vision'] = (dfnya['eyesight_right']+dfnya['eyesight_left'])/2

    #hearing_rata-rata
    dfnya['new_hearing'] = (dfnya['hearing_right']+dfnya['hearing_left'])/2

    #good cholesterol ratio
    dfnya['new_good_chol_ratio'] = (dfnya['hdl'] / dfnya['cholesterol'])*100

    #bad-good cholesterol ratio
    dfnya['new_bad_good_chol_ratio'] = (dfnya['ldl'] / dfnya['hdl'])*100

    #liverEnzimeRatio
    dfnya['new_liverEnzimeRatio'] = (dfnya['ast'] / dfnya['alt'])*100

    return dfnya


## calculate_vif(df, target_column='smoking', vif_threshold=5.0)

In [ ]:
def calculate_vif(df, target_column='smoking', vif_threshold=5.0):
    # Drop target
    X = df.drop(columns=[target_column])
    # Drop non-numeric columns if any (e.g., ID or categorical)
    X = X.select_dtypes(include='number')
    
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Compute VIF
    vif_data = pd.DataFrame()
    vif_data['feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
    
    # Sort by VIF descending
    vif_data = vif_data.sort_values(by='VIF', ascending=False).reset_index(drop=True)
    
    print('-------------------------------------------------------------')
    print(vif_data)
    print('-------------------------------------------------------------')
    print("High VIF features (possible multicollinearity):")
    print(vif_data[vif_data['VIF'] > vif_threshold])
    
    return vif_data


## remove_highly_correlated_features(df, target_column, threshold=0.7)

In [ ]:

def remove_highly_correlated_features(df, target_column, threshold=0.7):
    # Step 1: Calculate the correlation matrix
    correlation_matrix = df.corr()
    # print(f'correlation_matrix: {correlation_matrix}')
    
    # Step 2: Find pairs of features with correlation > threshold (absolute value)
    high_corr_pairs = correlation_matrix.unstack().sort_values(ascending=False)
    # print(f'Highly correlated pairs 1 :\n{high_corr_pairs}\n')

    high_corr_pairs = high_corr_pairs[high_corr_pairs.abs() > threshold]
    # print(f'Highly correlated pairs 2 :\n{high_corr_pairs}\n')
    
    # Step 3: Remove diagonal (self-correlations)
    high_corr_pairs = high_corr_pairs[high_corr_pairs < 1]
    print(f'Highly correlated pairs 3 :\n{high_corr_pairs}\n')
    
    # Step 4: Find feature importance with respect to the target variable
    target_corr = correlation_matrix[target_column].sort_values(ascending=False)
    print(f'Target variable correlation:\n{target_corr}\n')
    
    # Step 5: Iterate through the highly correlated pairs and remove less important feature
    features_to_remove = []
    for pair in high_corr_pairs.index:
        feature1, feature2 = pair
        # Remove the feature with lower correlation to the target
        if abs(correlation_matrix[feature1][target_column]) < abs(correlation_matrix[feature2][target_column]):
            features_to_remove.append(feature1)
        else:
            features_to_remove.append(feature2)
    
    # Remove the features that are less important (with lower correlation to the target)
    df.drop(columns=features_to_remove, inplace=True)
    
    return df, features_to_remove


## splitting_balancing_scaling(the_df, target_column):

In [ ]:
def splitting_balancing_scaling(the_df, target_column, df_name=''):
    # Ensure that index is consistent
    the_df.reset_index(drop=True, inplace=True)

    # SPLITTING
    print(f'Splitting the {df_name}')
    print('-----------------------------------')

    X = the_df.drop(columns=[target_column])
    y = the_df[target_column]

    # Perform train-test split, with stratification for target column
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    print("Shape of the_df:", the_df.shape)
    print("Shape of X:", X.shape)
    print("Shape of y:", y.shape)
    print("Shape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)
    print("===================================")

    # BALANCING
    print(f'Balancing training data')
    print('-----------------------------------')
    sm = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = sm.fit_resample(X_train, y_train)
    print("Original dataset shape:", X_train.shape, y_train.shape)
    print("Balanced dataset shape:", X_train_balanced.shape, y_train_balanced.shape)
    print(f"Original y_train:\n", y_train.value_counts(normalize=True))
    print(f"Balanced y_train:\n", y_train_balanced.value_counts(normalize=True))
    print("===================================")

    # SCALING
    print(f'Scaling X_train and X_test data')
    print('-----------------------------------')
    scaler = StandardScaler()
    X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)  # Fit and transform for training data
    X_test_scaled = scaler.transform(X_test)  # Only transform for test data

    return X_train_balanced_scaled, y_train_balanced, X_test_scaled, y_test, scaler

## drop_column_if_exists(df, column_name)

In [ ]:
def drop_column_if_exists(df, column_name):
    if column_name in df.columns:
        df.drop([column_name], axis=1, inplace=True)
        print(f"Column '{column_name}' dropped successfully.")
    else:
        print(f"Column '{column_name}' does not exist.")    
    return df

## drop_new_feature_yg_lemah_atau_feature_originalnya(df,listnya)

In [ ]:
def drop_new_feature_yg_lemah_atau_feature_originalnya(df,listnya):
    # fungsi ini membandingkan 2 feature pembentuk feature baru 
    # listnya adalah list dengan urutan FeatureLama1, FeatureLama2, FeatureBentukanBaru
    # jika feature baru justru lebih lemah maka feature baru dibuang saja (gak jadi dipakai)
    # jika feature baru paling tinggi , maka feature lama keduanya dibuang
    # jika salah satu feature saja yang lebih lemah dari feature baru, maka feature lemah itu yang dibuang

    corrnya=df.corr(numeric_only=True).loc['smoking', listnya]
    corrnya = corrnya.abs()
    print(f'{corrnya}\nthe lowest value => {corrnya.idxmin()} = {corrnya.min()}')
    if corrnya.idxmax() == listnya[2]:
        df=drop_column_if_exists(df,listnya[0])
        df=drop_column_if_exists(df,listnya[1])
    else:
        df=drop_column_if_exists(df,corrnya.idxmin())

    print('\n\n-------------------------------\n\n')    
    return df

## save_model(model, filename)

In [ ]:
def save_model(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {filename}")

## load_model(filename)

In [ ]:
def load_model(filename):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    print(f"Model loaded from {filename}")
    return model    

## load_model_and_predict(model_file_path,model_file_name, X_test_scaled,y_test)

In [ ]:
def load_model_and_predict(model_file_path,model_file_name, X_test_scaled,y_test):
    try:
        model = load_model(f'{model_file_path}{model_file_name}')
        y_pred = model.predict(X_test_scaled)
        print(f"== {model_file_name} ==")
        print(classification_report(y_test, y_pred))
        
    except ValueError as ve:
        print(f"ValueError: There was a mismatch in the data or model. Error details: {ve}")
    except AttributeError as ae:
        print(f"AttributeError: The model might be incompatible with the provided data. Error details: {ae}")
    except Exception as e:
        print(f"Unexpected error during prediction: {e}")

## hyper_parameter_tuning_LogisticReg_gridSearch_recall(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_LogisticReg_gridSearch_recall(X_train, y_train):

    # Parameter grid
    logreg_param_grid = {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear'],
        'clf__fit_intercept': [True, False],
        # 'clf__class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}]
        'clf__class_weight': [None, 'balanced']
    }

    # Pipeline (with class_weight)
    logreg_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
    ])

    # Custom scorer
    recall_scorer = make_scorer(recall_score, pos_label=1)

    # GridSearchCV
    logreg_grid = GridSearchCV(logreg_pipeline,
                            param_grid=logreg_param_grid,
                            scoring=recall_scorer,
                            cv=5,
                            verbose=1,
                            n_jobs=-1)

    logreg_grid.fit(X_train, y_train)

    best_logreg_model = logreg_grid.best_estimator_
    best_logreg_param = logreg_grid.best_params_
    print("Logistic Regression ; Grid Search ")
    
    print("Best Parameters:")
    for key, value in  logreg_grid.best_params_.items():
        print(f"{key}: {value}")
    
    print("--------------------------------------------")
    return best_logreg_model ,best_logreg_param

## hyper_parameter_tuning_LogisticReg_gridSearch_accuracy(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_LogisticReg_gridSearch_accuracy(X_train, y_train):
    # Parameter grid
    logreg_param_grid = {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear'],
        'clf__fit_intercept': [True, False],
        # 'clf__class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}] kalau data sudah balanced tidak butuh ini
        'clf__class_weight': [None, 'balanced']
        
    }

    # Pipeline
    logreg_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(max_iter=1000))
    ])

    # Use accuracy as scoring metric
    accuracy_scorer = make_scorer(accuracy_score)

    # GridSearchCV
    logreg_grid = GridSearchCV(
        estimator=logreg_pipeline,
        param_grid=logreg_param_grid,
        scoring=accuracy_scorer,
        cv=5,
        verbose=1,
        n_jobs=-1
    )

    logreg_grid.fit(X_train, y_train)

    best_logreg_model = logreg_grid.best_estimator_
    best_logreg_param = logreg_grid.best_params_

    print("Logistic Regression ; Grid Search (Accuracy Focused)")
    print("Best Parameters:")
    for key, value in best_logreg_param.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")

    return best_logreg_model, best_logreg_param


## hyper_parameter_tuning_LogisticReg_gridSearch_AUC(X_train, y_train)

In [ ]:

def hyper_parameter_tuning_LogisticReg_gridSearch_AUC(X_train, y_train):
    logreg_param_grid = {
        'clf__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'clf__penalty': ['l1', 'l2'],
        'clf__solver': ['liblinear'],  # 'liblinear' supports both l1 and l2
        'clf__fit_intercept': [True, False],
        'clf__class_weight': [None, 'balanced']
    }

    logreg_pipeline = Pipeline([
        # ('scaler', StandardScaler()),  # ⛔ Commented out for flexibility, enable if needed
        ('clf', LogisticRegression(max_iter=1000))
    ])

    logreg_grid = GridSearchCV(
        estimator=logreg_pipeline,
        param_grid=logreg_param_grid,
        scoring='roc_auc',  # ✅ Fokus ke AUC
        cv=5,
        verbose=1,
        n_jobs=-1
    )

    logreg_grid.fit(X_train, y_train)

    best_logreg_model = logreg_grid.best_estimator_
    best_logreg_param = logreg_grid.best_params_

    print("Logistic Regression ; Grid Search (AUC Focused)")
    print("Best Parameters:")
    for key, value in best_logreg_param.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")

    return best_logreg_model, best_logreg_param


## hyper_parameter_tuning_randomForest_randomSearch


| Parameter           | Description                                        |
| ------------------- | -------------------------------------------------- |
| `n_estimators`      | Number of trees in the forest                      |
| `max_depth`         | Maximum depth of each tree                         |
| `min_samples_split` | Minimum samples required to split a node           |
| `min_samples_leaf`  | Minimum samples at a leaf node                     |
| `max_features`      | Number of features to consider at each split       |
| `class_weight`      | Helps to boost recall for class 1 (same as before) |


## hyper_parameter_tuning_randomForest_randomSearch_recall(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_randomForest_randomSearch_recall(X_train, y_train):

    # Parameter search space
    rf_param_grid = {
        'clf__n_estimators': [100, 200, 300, 400],
        'clf__max_depth': [None, 10, 20, 30, 40],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__max_features': ['sqrt', 'log2'],
        # 'clf__class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}]
        'clf__class_weight': [None, 'balanced']
    }

    # Create pipeline
    rf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(random_state=42))
    ])

    # Focus on recall for class 1
    recall_scorer = make_scorer(recall_score, pos_label=1)

    # RandomizedSearchCV
    rf_search = RandomizedSearchCV(rf_pipeline,
                                    param_distributions=rf_param_grid,
                                    n_iter=50,
                                    scoring=recall_scorer,
                                    cv=5,
                                    verbose=1,
                                    n_jobs=-1,
                                    random_state=42)

    rf_search.fit(X_train, y_train)

    best_randomForest_model = rf_search.best_estimator_
    best_randomForest_param = rf_search.best_params_
    print("Random Forest ; Random Search ")
    print("Best Parameters:", best_randomForest_param)

    print("Best Parameters:")
    for key, value in  best_randomForest_param.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")

    return best_randomForest_model ,best_randomForest_param


## hyper_parameter_tuning_randomForest_randomSearch_accuracy(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_randomForest_randomSearch_accuracy(X_train, y_train):
    # Parameter search space
    rf_param_grid = {
        'clf__n_estimators': [100, 200, 300, 400],
        'clf__max_depth': [None, 10, 20, 30, 40],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__max_features': ['sqrt', 'log2'],
        # 'clf__class_weight': [None, 'balanced', {0: 1, 1: 2}, {0: 1, 1: 3}]
        'clf__class_weight': [None, 'balanced']

    }

    # Create pipeline
    rf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(random_state=42))
    ])

    # Accuracy scoring
    accuracy_scorer = make_scorer(accuracy_score)

    # RandomizedSearchCV
    rf_search = RandomizedSearchCV(
        estimator=rf_pipeline,
        param_distributions=rf_param_grid,
        n_iter=50,
        scoring=accuracy_scorer,
        cv=5,
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    rf_search.fit(X_train, y_train)

    best_randomForest_model = rf_search.best_estimator_
    best_randomForest_param = rf_search.best_params_

    print("Random Forest ; Random Search (Accuracy Focused)")
    print("Best Parameters:")
    for key, value in best_randomForest_param.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")

    return best_randomForest_model, best_randomForest_param


## hyper_parameter_tuning_randomForest_randomSearch_AUC(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_randomForest_randomSearch_AUC(X_train, y_train):
    rf_param_grid = {
        'clf__n_estimators': [100, 200, 300, 400],
        'clf__max_depth': [None, 10, 20, 30, 40],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4],
        'clf__max_features': ['sqrt', 'log2'],
        'clf__class_weight': [None, 'balanced']
    }

    rf_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', RandomForestClassifier(random_state=42))
    ])

    rf_search = RandomizedSearchCV(
        rf_pipeline,
        param_distributions=rf_param_grid,
        n_iter=50,
        scoring='roc_auc',  # GANTI di sini
        cv=5,
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    rf_search.fit(X_train, y_train)

    best_model = rf_search.best_estimator_
    best_params = rf_search.best_params_
    print("Random Forest ; Random Search (AUC Optimized)")
    print("Best Parameters:")
    for k, v in best_params.items():
        print(f"{k}: {v}")
    print("--------------------------------------------")

    return best_model, best_params


## hyper_parameter_tuning_XGB_randomSearch_recall(X_train, y_train)

| Parameter                 | Purpose                                                             |
| ------------------------- | ------------------------------------------------------------------- |
| `n_estimators`            | Number of boosting rounds (trees)                                   |
| `max_depth`               | Controls complexity (overfitting risk if too deep)                  |
| `learning_rate`           | Smaller values = slower, more accurate learning                     |
| `subsample`               | % of samples used per tree (helps generalization)                   |
| `colsample_bytree`        | % of features used per tree                                         |
| `gamma`                   | Minimum loss reduction to make a split (higher = more conservative) |
| `reg_alpha`, `reg_lambda` | L1/L2 regularization (controls overfitting)                         |
| `scale_pos_weight`        | Helps with imbalanced classes — gives class 1 more "push"           |


In [ ]:
def hyper_parameter_tuning_XGB_randomSearch_recall(X_train, y_train):
    xgb_param_grid = {
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [3, 5, 7, 10],
        'clf__learning_rate': [0.01, 0.1, 0.2],
        'clf__subsample': [0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.6, 0.8, 1.0],
        'clf__gamma': [0, 1, 5],
        'clf__reg_alpha': [0, 0.1, 1],
        'clf__reg_lambda': [0.5, 1, 5],
        # 'clf__scale_pos_weight': [1, 2, 3]  # balances positive class (helpful for class 1 recall)
        #clf__scale_pos_weight tidak dibutuhkan kalau data sudah balanced
    }

    xgb_pipeline = Pipeline([
        ('scaler', StandardScaler()),  # optional, but safe to include
        ('clf', XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42))
    ])

    recall_scorer = make_scorer(recall_score, pos_label=1)

    xgb_random_search = RandomizedSearchCV(xgb_pipeline,
                                    param_distributions=xgb_param_grid,
                                    n_iter=50,
                                    scoring=recall_scorer,
                                    cv=5,
                                    verbose=1,
                                    n_jobs=-1,
                                    random_state=42)
    xgb_random_search.fit(X_train, y_train)
    print("XGBoost ; Random Search ")
    print("Best Parameters:")
    for key, value in  xgb_random_search.best_params_.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")    
    
    return xgb_random_search.best_estimator_ , xgb_random_search.best_params_


##  hyper_parameter_tuning_XGB_randomSearch_accuracy(X_train, y_train)

In [ ]:

def hyper_parameter_tuning_XGB_randomSearch_accuracy(X_train, y_train):
    xgb_param_grid = {
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [3, 5, 7, 10],
        'clf__learning_rate': [0.01, 0.1, 0.2],
        'clf__subsample': [0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.6, 0.8, 1.0],
        'clf__gamma': [0, 1, 5],
        'clf__reg_alpha': [0, 0.1, 1],
        'clf__reg_lambda': [0.5, 1, 5],
        # 'clf__scale_pos_weight': [1, 2, 3] # #clf__scale_pos_weight tidak dibutuhkan kalau data sudah balanced
    }

    xgb_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('clf', XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42))
    ])

    accuracy_scorer = make_scorer(accuracy_score)

    xgb_random_search = RandomizedSearchCV(
        estimator=xgb_pipeline,
        param_distributions=xgb_param_grid,
        n_iter=50,
        scoring=accuracy_scorer,
        cv=5,
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    xgb_random_search.fit(X_train, y_train)

    print("XGBoost ; Random Search (Accuracy Focused)")
    print("Best Parameters:")
    for key, value in xgb_random_search.best_params_.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")    

    return xgb_random_search.best_estimator_, xgb_random_search.best_params_


## hyper_parameter_tuning_XGB_randomSearch_AUC(X_train, y_train)

In [ ]:
def hyper_parameter_tuning_XGB_randomSearch_AUC(X_train, y_train):
    xgb_param_grid = {
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [3, 5, 7, 10],
        'clf__learning_rate': [0.01, 0.1, 0.2],
        'clf__subsample': [0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.6, 0.8, 1.0],
        'clf__gamma': [0, 1, 5],
        'clf__reg_alpha': [0, 0.1, 1],
        'clf__reg_lambda': [0.5, 1, 5],
        # 'clf__scale_pos_weight': [1, 2, 3]  # Uncomment if data imbalance is an issue
    }

    xgb_pipeline = Pipeline([
        # ('scaler', StandardScaler()),  # ⛔ Commented: XGBoost usually handles feature scale well
        ('clf', XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42))
    ])

    xgb_random_search = RandomizedSearchCV(
        estimator=xgb_pipeline,
        param_distributions=xgb_param_grid,
        n_iter=50,
        scoring='roc_auc',  # ✅ Ganti scoring ke AUC
        cv=5,
        verbose=1,
        n_jobs=-1,
        random_state=42
    )

    xgb_random_search.fit(X_train, y_train)

    print("XGBoost ; Random Search (AUC Focused)")
    print("Best Parameters:")
    for key, value in xgb_random_search.best_params_.items():
        print(f"{key}: {value}")
    print("--------------------------------------------")    

    return xgb_random_search.best_estimator_, xgb_random_search.best_params_


## add_to_performance_report_comparison( model_name,y_test, y_pred, X_test,listBaru=False,scaler_name='', dataframe_name='',best_params='', tuning_focus='', verbose=False)

In [ ]:
def calculate_auc(model, X_test, y_test):
    y_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    return auc

In [ ]:
def add_to_performance_report_comparison( model_name,y_test, y_pred, X_test, listBaru=False,scaler_name='', dataframe_name='',best_params='', tuning_focus='', verbose=False):
    try:
        df_kompilasihasil=pd.read_excel(f'./output/dataframe_kompilasi_hasil_modelling.xlsx', index_col=None)
        df_kompilasihasil = df_kompilasihasil.loc[:, ~df_kompilasihasil.columns.str.startswith('Unnamed')]

    except Exception as e:
        print("Error reading the dataframe_kompilasi_hasil_modelling.xlsx :", str(e))
        df_kompilasihasil = None

    model = load_model(f'./output/{model_name}') # untuk kebutuhan menghitung AUC

    performance = {
        'Model': [model_name],
        'Accuracy': [accuracy_score(y_test, y_pred)],
        'Precision': [precision_score(y_test, y_pred)],
        'Recall': [recall_score(y_test, y_pred)],
        'F1 Score': [f1_score(y_test, y_pred)],
        'RoC-AuC': [calculate_auc(model, X_test, y_test)],
        'scaler_name': [scaler_name],
        'dataframe_name': [dataframe_name],
        'tuning_focus': [tuning_focus],
        'best_params': [str(best_params)]
    }
    tempDF = pd.DataFrame(performance)
    if verbose :
        print(tempDF)
    
    if listBaru == True:
        result= tempDF
    else:
        if df_kompilasihasil is None:
            result= tempDF
        else:
            result =pd.concat([df_kompilasihasil, tempDF], axis=0)

    result = result.drop_duplicates()
        
    result.to_excel(f'./output/dataframe_kompilasi_hasil_modelling.xlsx', index=False)
    
    return result

## plot_roc_curves_for_each_test(models_with_data: dict):

In [ ]:
def plot_roc_curves_for_each_test(models_with_data: dict,savedImageName):
    plt.figure(figsize=(8, 6))

    for name, (model, X_test, y_test) in models_with_data.items():
        if hasattr(model, "predict_proba"):
            y_proba = model.predict_proba(X_test)[:, 1]
        else:
            raise ValueError(f"Model '{name}' does not support predict_proba().")

        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc_score = roc_auc_score(y_test, y_proba)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc_score:.3f})")

    plt.plot([0, 1], [0, 1], "k--", label="Random (AUC = 0.5)")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate (Recall)")
    plt.title("ROC Curve Comparison")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(savedImageName, dpi=300)
    plt.show()


# EDA , Data Cleansing and Preprocessing

## Create folder for output files

buat tempat untuk menyimpan file output

output files will be used for :

- untuk double checking
- for documentation 
- pickle files from model training result will be used for production


In [ ]:
folder_name = 'output'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Folder '{folder_name}' already exists.")

## Info Dasar

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.str.replace("(","_",regex=False)
df.columns = df.columns.str.replace(")","",regex=False)
df.columns = df.columns.str.replace(" ","_",regex=False)
df.columns = df.columns.str.lower()
df.columns

In [ ]:
daftarKolomOriginal = df.columns
daftarKolomOriginal

In [ ]:
numeric_cols = df.select_dtypes(include='number').columns.tolist()
numeric_cols

In [ ]:
df_head = df.head()
df_info = df.dtypes
df_shape = df.shape


In [ ]:
df_head

In [ ]:
df_info

In [ ]:
df_shape

In [ ]:
df.describe().T


In [ ]:
num_duplicates = df.duplicated().sum()
print(f"Jumlah duplikasi: {num_duplicates}")    

In [ ]:
duplikacted_rows = df[df.duplicated(keep=False)].sort_values(list(df.columns))
print(f"duplikacted_rows:\n{duplikacted_rows}")
duplikacted_rows.to_excel(r'.\output\xls_01_duplicated_rows.xlsx', index=False) # disimpan ke CSV for future checking


## remove duplcate Row 

In [ ]:
df_shape = df.shape
print(f"Shape = {df_shape}")

df.drop_duplicates(inplace=True)

df_shape = df.shape
print(f"Shape after removing duplicates: {df_shape}")


removing duplicates 🟢
checing for null values 🟢


## check missing value 


In [ ]:
df.isnull().any().any()

## Mini EDA

In [ ]:
mini_eda(df,r'.\output\pdf_01_mini_eda_result.pdf')

## backup original dataframe


In [ ]:
df_000_original = df.copy(deep=True)

## capping / flagging by Impossible Values

In [ ]:
df = df_000_original.copy(deep=True)

In [ ]:
# df.to_excel('./output/xls_02_smokerStatus_check_impossible_value_before.xlsx', index=False)

In [ ]:
impossible_thresholds = pd.DataFrame([
    {'feature': 'age', 'min': 0, 'max': 120},
    {'feature': 'height_cm', 'min': 120, 'max': 210},
    {'feature': 'weight_kg', 'min': 35, 'max': 200},
    {'feature': 'waist_cm', 'min': 50, 'max': 150},
    {'feature': 'eyesight_left', 'min': 0.1, 'max': 2.0},
    {'feature': 'eyesight_right', 'min': 0.1, 'max': 2.0},
    {'feature': 'hearing_left', 'min': 0, 'max': 2},
    {'feature': 'hearing_right', 'min': 0, 'max': 2},
    {'feature': 'systolic', 'min': 90, 'max': 180},
    {'feature': 'relaxation', 'min': 60, 'max': 120},
    {'feature': 'fasting_blood_sugar', 'min': 70, 'max': 126},
    {'feature': 'cholesterol', 'min': 125, 'max': 200},
    {'feature': 'triglyceride', 'min': 50, 'max': 150},
    {'feature': 'hdl', 'min': 40, 'max': 100},
    {'feature': 'ldl', 'min': 50, 'max': 130},
    {'feature': 'hemoglobin', 'min': 12.0, 'max': 17.5},
    {'feature': 'urine_protein', 'min': 0, 'max': 4},
    {'feature': 'serum_creatinine', 'min': 0.6, 'max': 1.3},
    {'feature': 'ast', 'min': 8, 'max': 33},
    {'feature': 'alt', 'min': 7, 'max': 56},
    {'feature': 'gtp', 'min': 8, 'max': 61},
    {'feature': 'dental_caries', 'min': 0, 'max': 1},
])

df = capping_features_based_on_threshold(df,impossible_thresholds)

In [ ]:
# df.to_excel('./output/xls_03_smokerStatus_check_impossible_value_after.xlsx', index=False)

In [ ]:
# mini_eda(df,r'.\output\pdf_02_mini_eda_result_after_cap_threshold.pdf')

## Check Target immbalance

In [ ]:
df['smoking'].value_counts(normalize=False)

In [ ]:
df['smoking'].value_counts(normalize=True)
#normalize = True (untuk tau distribusi dalam percent, bukan cuma jumlah count nya)

## boxplots and distribution plots

In [ ]:
counter = 0
for column in daftarKolomOriginal:
    counter += 1  
    
    if column in df.columns:
        lower_bound, upper_bound, num_outliers, percent_outliers, above_upper, below_lower = calculate_IQR(df, column)
        
        fig, axes = plt.subplots(1, 2, figsize=(9, 3))
        
        sns.boxplot(data=df[column], ax=axes[0])
        axes[0].set_title(f"Boxplot of {column}")
        axes[0].axhline(y=lower_bound, color='r', linestyle='--', label=f'Lower Bound: {lower_bound}')
        axes[0].axhline(y=upper_bound, color='g', linestyle='--', label=f'Upper Bound: {upper_bound}')
        
        text_str = f"Lower Bound: {lower_bound}\nUpper Bound: {upper_bound}\n" \
                   f"Outliers: {num_outliers}\n% Outliers: {percent_outliers:.2f}%\n" \
                   f"Above Upper: {above_upper}\nBelow Lower: {below_lower}"
        
        axes[0].text(1.5, 0.5, text_str, transform=axes[0].transAxes,
                     fontsize=10, verticalalignment='center', horizontalalignment='left',
                     bbox=dict(facecolor='white', alpha=0.7, edgecolor='black', boxstyle='round,pad=1'))
        
        sns.histplot(df[column], kde=True, bins=50, ax=axes[1])
        axes[1].set_title(f"Distribution of {column}")
        axes[1].set_xlabel(column)
        axes[1].set_ylabel("Count")
        
        plt.tight_layout()
        
        plt.savefig(f'./output/plot_01_boxplot_and_distributionplot_{str(counter).zfill(2)}_of_{column}.jpg', dpi=300)
        
        plt.show()

    else:
        print(f"Column '{column}' not found in the DataFrame.")


## Check for NULL values

In [ ]:
df.isnull().any().any()

# Balancing before feature Engineering

## add combined feature

In [ ]:
df_M2_step01 = df_000_original.copy(deep=True)
df_M2_step01 = add_combined_features(df_M2_step01)

## Splitting_balancing_scaling 

In [ ]:
X_train,y_train, X_test, y_test,M2_scalernya = splitting_balancing_scaling(df_M2_step01, 'smoking',df_name ='df_M2_step01' )

save_model(M2_scalernya,f'./output/model_M2_scalernya.pkl')


X_train_df =  pd.DataFrame(X_train)
X_train_df.columns = df_M2_step01.drop(['smoking'], axis=1).columns
X_train_df.reset_index(drop=True, inplace=True) 
y_train_df = pd.Series(y_train, name='smoking') 
y_train_df.reset_index(drop=True, inplace=True) 
df_M2_step02_train = pd.concat([X_train_df, y_train_df], axis=1)



X_test_df =  pd.DataFrame(X_test)
X_test_df.columns = df_M2_step01.drop(['smoking'], axis=1).columns
X_test_df.reset_index(drop=True, inplace=True) 
y_test_df = pd.Series(y_test, name='smoking') 
y_test_df.reset_index(drop=True, inplace=True)
df_M2_step02_test = pd.concat([X_test_df, y_test_df], axis=1)


In [ ]:
print(df_M2_step01.shape)
print(df_M2_step02_train.shape)
print(df_M2_step02_test.shape)
nan_rows = df_M2_step02_test[df_M2_step02_test.isna().any(axis=1)]
print(nan_rows)

In [ ]:
df_M2_step02_train_corr = df_M2_step02_train.corr(numeric_only=True)
show_corrMap(df_M2_step02_train_corr,'./output/plot_04_correlationMap_df_M2_step02_train_corr.jpg')

## Check whether new feature is good or bad


In [ ]:
# menggunakan fungsi : def drop_new_feature_yg_lemah_atau_feature_originalnya(df,listnya):
    # fungsi ini membandingkan 2 feature pembentuk feature baru 
    # listnya adalah list dengan urutan FeatureLama1, FeatureLama2, FeatureBentukanBaru
    # jika feature baru justru lebih lemah maka feature baru dibuang saja (gak jadi dipakai)
    # jika feature baru paling tinggi , maka feature lama keduanya dibuang
    # jika salah satu feature saja yang lebih lemah dari feature baru, maka feature lemah itu yang dibuang
    

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['weight_kg', 'height_cm', 'new_bmi'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['waist_cm', 'height_cm', 'new_waist_height_ratio'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['systolic', 'relaxation', 'new_pulse_preasure'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['eyesight_right', 'eyesight_left', 'new_vision'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['hearing_right', 'hearing_left', 'new_hearing'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['hdl', 'cholesterol', 'new_good_chol_ratio'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['ldl', 'hdl', 'new_bad_good_chol_ratio'])

df_M2_step02_train=drop_new_feature_yg_lemah_atau_feature_originalnya(df_M2_step02_train,['ast', 'alt', 'new_liverEnzimeRatio'])



## Feature Selection 

In [ ]:
df_M2_step02_train.columns

In [ ]:
df_M2_step02_train_corr = df_M2_step02_train.corr(numeric_only=True)
show_corrMap(df_M2_step02_train_corr,'./output/plot_04_correlationMap_df_M2_step02_train_corr.jpg')

### drop feature yang correlation lemah < 0.05

In [ ]:
selection_threshold = 0.05 # correlation threshold
selected_features_prep=df_M2_step02_train_corr['smoking']
print(f'sebelum dipilih=\n{selected_features_prep}')

In [ ]:
selected_features = selected_features_prep[abs(selected_features_prep) > selection_threshold]
print('')
print(f'selected_featres=\n{selected_features}')


In [ ]:
selected_feature_names = selected_features.index.to_list()
selected_feature_names

### remove feature with high Multicoliniearity

In [ ]:
df_M2_step03_train = df_M2_step02_train[selected_feature_names]

In [ ]:
vif_result = calculate_vif(df_M2_step03_train,'smoking', 5)

In [ ]:

df_M2_step03_train.drop(['new_good_chol_ratio'], axis=1, inplace=True)
vif_result = calculate_vif(df_M2_step03_train,'smoking', 5)

In [ ]:
df_M2_step03_train.drop(['weight_kg'], axis=1, inplace=True)
vif_result = calculate_vif(df_M2_step03_train,'smoking', 5)

## remove features with high correlation betwen features (>0.75)

remove the one that has weaker correlation with target

In [ ]:
target_column = 'smoking' 
df_M2_step04_train, removed_features = remove_highly_correlated_features(df_M2_step03_train, target_column, threshold=0.7)

# Baseline Modelling

In [ ]:
# UNCOMENT SALAH SATU BLOK DI BAWAHA INI UNTUK MEMILIH DATA DARI DATA FRAME MANA YANG MAU PAKAI UNTUK MODELLING

#=====================================================
# the_df = df_000_original.select_dtypes(include='number').copy(deep=True)
# nama_dataframe_asal= 'df_000_original_155rb'
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# scaler_name = f'./output/model_{nama_dataframe_asal}_scalernya.pkl'
# save_model(scalernya, f'./output/model_{nama_dataframe_asal}_scalernya.pkl')


# #=====================================================
the_df = df_M2_step04_train.copy(deep=True)
nama_dataframe_asal= 'df_M2_step04_train'
selected_column =list(df_M2_step04_train.columns)
for col in df_M2_step02_test.columns:
    if col not in df_M2_step04_train:
        df_M2_step02_test.drop(columns=col, inplace=True)
X_train = df_M2_step04_train.drop(columns='smoking')
y_train = df_M2_step04_train['smoking']
X_test = df_M2_step02_test.drop(columns='smoking')
y_test = df_M2_step02_test['smoking']
scaler_name = 'model_M2_scalernya.pkl'
# ======================================================================================================



In [ ]:
target_column = 'smoking'

print(f'===================================================')
print(".")
print(".")

# Logistic Regression
modelnya = LogisticRegression()
modelnya.fit(X_train, y_train)
model_name = f'model_{nama_dataframe_asal}_logreg_basic.pkl'
model_file_full_path = f'./output/{model_name}'
save_model(modelnya, model_file_full_path)
model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
# print(f"== {model_name} ==")
# print(classification_report(y_test, y_pred))
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal)

model_logregr=modelnya

# Random Forest Classifier
modelnya = RandomForestClassifier()
modelnya.fit(X_train, y_train)
model_name = f'model_{nama_dataframe_asal}_randomForest_basic.pkl'
model_file_full_path = f'./output/{model_name}'
save_model(modelnya, model_file_full_path)
model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
# print(f"== {model_name} ==")
# print(classification_report(y_test, y_pred))
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal)

model_rndfore=modelnya



# XGBoost Classifier
modelnya = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
modelnya.fit(X_train, y_train)
model_name = f'model_{nama_dataframe_asal}_xgbost_basic.pkl'
model_file_full_path = f'./output/{model_name}'
save_model(modelnya, model_file_full_path)
model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
# print(f"== {model_name} ==")
# print(classification_report(y_test, y_pred))
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal)

model_xgboost=modelnya



models_with_data = {
    "LogRegr": (model_logregr, X_test, y_test),
    "RndFore": (model_rndfore, X_test, y_test),
    "XGBoost": (model_xgboost, X_test, y_test),
}
plot_roc_curves_for_each_test(models_with_data,f'./output/plot_10_ROC_Curve_model_df_M2_step04_train_gabungan_basic.jpg')

In [ ]:
df_kompilasihasil.sort_values(by='Recall', ascending=False)

| **Metric**              | **Why it matters**                                                                                                                                                                                |
| ----------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Recall (class 1)**    | Measures how many real smokers the model correctly detects. <br>Use when it's critical not to miss any smokers. Example: in medical screening, you want to identify everyone who needs attention. |
| **F1-score (class 1)**  | Combines both precision and recall. <br>Use this when you want a **balanced performance** — not too many false negatives, and not too many false positives.                                       |
| **Precision (class 1)** | Measures how many of the predicted smokers are actually smokers. <br>Use this when **predicting someone as a smoker has serious consequences** (e.g., legal, privacy, or social impact).          |


# Hyper Parameter Tuning (Focus on Recall)

In [ ]:
# UNCOMENT SALAH SATU BLOK DI BAWAHA INI UNTUK MEMILIH DATA DARI DATA FRAME MANA YANG MAU PAKAI UNTUK MODELLING

#=====================================================
# the_df = df_000_original.select_dtypes(include='number').copy(deep=True)
# nama_dataframe_asal= 'df_000_original_155rb'
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# scaler_name = f'./output/model_{nama_dataframe_asal}_scalernya.pkl'
# save_model(scalernya, f'./output/model_{nama_dataframe_asal}_scalernya.pkl')

the_df = df_M2_step04_train.copy(deep=True)
nama_dataframe_asal= 'df_M2_step04_train'
selected_column =list(df_M2_step04_train.columns)
for col in df_M2_step02_test.columns:
    if col not in df_M2_step04_train:
        df_M2_step02_test.drop(columns=col, inplace=True)
X_train = df_M2_step04_train.drop(columns='smoking')
y_train = df_M2_step04_train['smoking']
X_test = df_M2_step02_test.drop(columns='smoking')
y_test = df_M2_step02_test['smoking']
scaler_name = 'model_M2_scalernya.pkl'
# ======================================================================================================

## GridSearch for Logistic Regression

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_LogisticReg_gridSearch_recall(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_logreg_bestModel_recall.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)


model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Recall')

## Random Search for Random Forest

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_randomForest_randomSearch_recall(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_randomForest_bestModel_recall.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Recall')


## RandomSearch for XGBoost 

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_XGB_randomSearch_recall(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_xgboost_bestModel_recall.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Recall')

# Hyper Parameter Tuning (Focuks on Accuracy)

In [ ]:
# UNCOMENT SALAH SATU BLOK DI BAWAHA INI UNTUK MEMILIH DATA DARI DATA FRAME MANA YANG MAU PAKAI UNTUK MODELLING

#=====================================================
# the_df = df_000_original.select_dtypes(include='number').copy(deep=True)
# nama_dataframe_asal= 'df_000_original_155rb'
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# scaler_name = f'./output/model_{nama_dataframe_asal}_scalernya.pkl'
# save_model(scalernya, f'./output/model_{nama_dataframe_asal}_scalernya.pkl')

the_df = df_M2_step04_train.copy(deep=True)
nama_dataframe_asal= 'df_M2_step04_train'
selected_column =list(df_M2_step04_train.columns)
for col in df_M2_step02_test.columns:
    if col not in df_M2_step04_train:
        df_M2_step02_test.drop(columns=col, inplace=True)
X_train = df_M2_step04_train.drop(columns='smoking')
y_train = df_M2_step04_train['smoking']
X_test = df_M2_step02_test.drop(columns='smoking')
y_test = df_M2_step02_test['smoking']
scaler_name = 'model_M2_scalernya.pkl'
# ======================================================================================================

## GridSearch for Logistic Regression

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_LogisticReg_gridSearch_accuracy(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_logreg_bestModel_Accuracy.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Accuracy')

## RandomSearch for RandomForest

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_randomForest_randomSearch_accuracy(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_randomForest_bestModel_Accuracy.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)


model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Accuracy')

## RandomSearch for XGBoost

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_XGB_randomSearch_accuracy(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_xgboost_bestModel_Accuracy.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for Accuracy')

# Hyper Parameter Tuning (Focus on AUC)

In [ ]:
# UNCOMENT SALAH SATU BLOK DI BAWAHA INI UNTUK MEMILIH DATA DARI DATA FRAME MANA YANG MAU PAKAI UNTUK MODELLING

#=====================================================
# the_df = df_000_original.select_dtypes(include='number').copy(deep=True)
# nama_dataframe_asal= 'df_000_original_155rb'
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# X_train,y_train, X_test, y_test,scalernya = splitting_balancing_scaling(the_df, target_column, nama_dataframe_asal)
# scaler_name = f'./output/model_{nama_dataframe_asal}_scalernya.pkl'
# save_model(scalernya, f'./output/model_{nama_dataframe_asal}_scalernya.pkl')


the_df = df_M2_step04_train.copy(deep=True)
nama_dataframe_asal= 'df_M2_step04_train'
selected_column =list(df_M2_step04_train.columns)
for col in df_M2_step02_test.columns:
    if col not in df_M2_step04_train:
        df_M2_step02_test.drop(columns=col, inplace=True)
X_train = df_M2_step04_train.drop(columns='smoking')
y_train = df_M2_step04_train['smoking']
X_test = df_M2_step02_test.drop(columns='smoking')
y_test = df_M2_step02_test['smoking']
scaler_name = 'model_M2_scalernya.pkl'
# ======================================================================================================

## GridSearch for Logistic Regression

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_LogisticReg_gridSearch_AUC(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_logreg_bestModel_AUC.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for AUC')

In [ ]:
models_with_data = {
    "LogReg": (model, X_test, y_test),
}
plot_roc_curves_for_each_test(models_with_data,f'./output/plot_10_ROC_AUC_Curve_{model_name}.jpg')

In [ ]:
model_logreg=model

## RandomSearch for RandomForest

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_randomForest_randomSearch_AUC(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_randomForest_bestModel_AUC.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)


model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for AUC')

In [ ]:
models_with_data = {
    "RandomForest": (model, X_test, y_test),
}
plot_roc_curves_for_each_test(models_with_data,f'./output/plot_10_ROC_AUC_Curve_{model_name}.jpg')

In [ ]:
model_rf=model

## RandomSearch for XGBoost

In [ ]:
best_model,best_model_params = hyper_parameter_tuning_XGB_randomSearch_AUC(X_train, y_train)
# best_model.fit(X_train, y_train)

model_name = f'model_{nama_dataframe_asal}_xgboost_bestModel_AUC.pkl'
model_file_full_path = f'./output/{model_name}'

save_model(best_model,model_file_full_path)

model = load_model(model_file_full_path)
y_pred = model.predict(X_test)
df_kompilasihasil=add_to_performance_report_comparison( model_name,y_test, y_pred,X_test,False,scaler_name,nama_dataframe_asal,best_model_params,'Tuned for AUC')

In [ ]:
models_with_data = {
    "XGBoost": (model, X_test, y_test),
}
plot_roc_curves_for_each_test(models_with_data,f'./output/plot_10_ROC_AUC_Curve_{model_name}.jpg')

In [ ]:
model_xgb=model

In [ ]:
models_with_data = {
    "LogisticReg": (model_logreg, X_test, y_test),
    "RandomForest": (model_rf, X_test, y_test),
    "XGBoost": (model_xgb, X_test, y_test)
}
plot_roc_curves_for_each_test(models_with_data,f'./output/plot_10_ROC_AUC_Curve_Gabungan_AUC.jpg')

# Selecting Best Modelling for Production

In [ ]:
df_kompilasihasil_sorted = df_kompilasihasil.sort_values(by='RoC-AuC', ascending=False)
df_kompilasihasil_sorted
